In [78]:
import numpy as np
import quantities as pq
import neo
import pickle

matToCsv.m outputs 1 csv file per trial that contains the spiking data with shape (neurons,time)

This script creates a list of dictionaries from the csv files in the form:

`trials = [{ID: 1, y: y_1},...,{ID: N, y: y_N}]`

Then puts `trials` in the form needed for elephant's GPFA implementation

In [19]:
## load trials into list of dictionaries

numTrials = 1877
numNeurons = 23

trials = []
for i in range(1,numTrials+1):
    filename = 'data/trial' + str(i) + '.csv'
    trials.append( {'ID': i, 'y': pd.read_csv(filename,header=None).to_numpy()}   )


Save trials to pickle file

In [24]:
with open('data/trials.pickle', 'wb') as filename:
    pickle.dump(trials, filename)

Load pickle file

In [33]:
with open('data/trials.pickle', 'rb') as filename:
    trials = pickle.load(filename)
    
numTrials = len(trials)
len(trials)

1877

Put data in correct form for elephant_gpfa. It wants neo.SpikeTrain objects that hold arrays of spike times:

`spiketrains = [[SpikeTrain,...,SpikeTrain],...,[SpikeTrain,...,SpikeTrain]]`

where the len(spiketrains) = number of trials and len(spiketrains[trial]) = number of neurons

First thing to do is to create spike time arrays

In [79]:
def singleNeuronSpikeTrain(neuron):
    # return neo.SpikeTrain object for a single trial and single neuron
    times = []
    for i in range(len(neuron)):
        if neuron[i] == 1:
            times.append(i)
    
    return neo.SpikeTrain(times=times, units='ms',t_stop=1603)
    

def singleTrialSpikeTrain(trial):
    # return array of neo.SpikeTrain objects for a single trial and all neurons
    st_array = []
    for i in range(numNeurons):
        st_array.append(singleNeuronSpikeTrain(trial[i]))
    
    return st_array


def fullSpikeTrain(trials):
    nested_st_array = []
    for i in range(numTrials):
        nested_st_array.append(singleTrialSpikeTrain(trials[i]['y']))
    
    return nested_st_array


nested_st_array = fullSpikeTrain(trials)

Let's save nested_st_array to a pickle now. This nested array contains SpikeTrain objects that contain spike times

In [80]:
with open('data/spikeTimes.pickle', 'wb') as filename:
    pickle.dump(nested_st_array, filename)

And to load it:

In [81]:
with open('data/spikeTimes.pickle', 'rb') as filename:
    st = pickle.load(filename)

In [82]:
print(st[0][0])

[ 224.  721.  732.  758.  775.  780.  790.  814.  818.  830.  842.  846.
  866.  873.  880.  888.  900.  921.  931.  959.  961.  981. 1003. 1005.
 1032. 1038. 1063. 1098. 1107. 1134. 1147. 1173. 1193. 1212. 1242. 1262.
 1281. 1296. 1399. 1481.] ms
